In [17]:
endpoint = "https://rita.cognitiveservices.azure.com/"
key = "b722c0c73b1f4ecba888774b52a72b9d"

In [1]:
"""
This code sample shows Prebuilt Read operations with the Azure Form Recognizer client library. 
The async versions of the samples require Python 3.6 or later.

To learn more, please visit the documentation - Quickstart: Document Intelligence (formerly Form Recognizer) SDKs
https://learn.microsoft.com/azure/ai-services/document-intelligence/quickstarts/get-started-sdks-rest-api?pivots=programming-language-python
"""

from azure.core.credentials import AzureKeyCredential
from azure.ai.formrecognizer import DocumentAnalysisClient

"""
Remember to remove the key from your code when you're done, and never post it publicly. For production, use
secure methods to store and access your credentials. For more information, see 
https://docs.microsoft.com/en-us/azure/cognitive-services/cognitive-services-security?tabs=command-line%2Ccsharp#environment-variables-and-application-configuration
"""

filePath = r"C:\Users\ediso\OneDrive\Desktop\ibm-cfc-2024\rita-cfc-2024\ai\course-prep\textbook-pdf\康軒數學5上教冊資料篇_全.pdf"

def format_bounding_box(bounding_box):
    if not bounding_box:
        return "N/A"
    return ", ".join(["[{}, {}]".format(p.x, p.y) for p in bounding_box])

def analyze_read():
    # sample document
    # formUrl = "https://raw.githubusercontent.com/Azure-Samples/cognitive-services-REST-api-samples/master/curl/form-recognizer/sample-layout.pdf"

    document_analysis_client = DocumentAnalysisClient(
        endpoint=endpoint, credential=AzureKeyCredential(key)
    )
    
    # poller = document_analysis_client.begin_analyze_document_from_url(
    #         "prebuilt-read", formUrl)
    
    with open(filePath, "rb") as file:
        poller = document_analysis_client.begin_analyze_document(
            "prebuilt-read", document=file
        )

    result = poller.result()

    print ("Document contains content: ", result.content)
    
    for idx, style in enumerate(result.styles):
        print(
            "Document contains {} content".format(
                "handwritten" if style.is_handwritten else "no handwritten"
            )
        )

    for page in result.pages:
        print("----Analyzing Read from page #{}----".format(page.page_number))
        print(
            "Page has width: {} and height: {}, measured with unit: {}".format(
                page.width, page.height, page.unit
            )
        )

        for line_idx, line in enumerate(page.lines):
            print(
                "...Line # {} has text content '{}' within bounding box '{}'".format(
                    line_idx,
                    line.content,
                    format_bounding_box(line.polygon),
                )
            )

        for word in page.words:
            print(
                "...Word '{}' has a confidence of {}".format(
                    word.content, word.confidence
                )
            )

    print("----------------------------------------")
    return result


if __name__ == "__main__":
    result = analyze_read()


In [13]:
textbook_texts = result.content
save_path = r"C:\Users\ediso\OneDrive\Desktop\ibm-cfc-2024\rita-cfc-2024\ai\course-prep\textbook-extracted\azure_document_intelligence"
save_file_name = "kang_math_5th_1st_extracted.txt"

with open(rf'{save_path}\{save_file_name}', 'w', encoding='utf-8') as txt_file:
    txt_file.write(textbook_texts)